# RSD Demo — Regime Shift Diagnostic

This notebook demonstrates how RSD detects **structural regime changes**
in a time series without forecasting future values.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)


## Concept

A regime shift occurs when the **structure** of the system changes,
not necessarily its mean value.

We detect this by monitoring local gradient and curvature energy.


In [ ]:
N = 4000
t = np.linspace(0, 40, N)

# Regime 1
sig1 = np.sin(t[:2000]) + 0.1 * np.random.randn(2000)

# Regime 2 (structural change)
sig2 = 0.5 * np.sin(3 * t[2000:]) + 0.3 * np.random.randn(2000)

signal = np.concatenate([sig1, sig2])


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(signal)
plt.axvline(2000, color="red", linestyle="--", label="True regime change")
plt.legend()
plt.title("Time Series with Hidden Regime Shift")
plt.show()


## Structural diagnostic

We compute a local structural score based on:
- gradient energy
- curvature energy

Sudden changes indicate regime instability.


In [ ]:
dx = np.diff(signal, prepend=signal[0])
ddx = np.diff(dx, prepend=dx[0])

grad_energy = dx**2
curv_energy = ddx**2

score = (
    grad_energy / (grad_energy.mean() + 1e-8)
    + curv_energy / (curv_energy.mean() + 1e-8)
)

score = score / score.max()


In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,6), sharex=True)

ax[0].plot(signal, label="Signal")
ax[0].axvline(2000, color="red", linestyle="--")
ax[0].set_title("Signal")

ax[1].plot(score, color="crimson", label="RSD score")
ax[1].axvline(2000, color="red", linestyle="--")
ax[1].set_title("Structural Regime Diagnostic")

for a in ax:
    a.legend()

plt.show()


## Interpretation

- Stable score → stable regime
- Persistent increase → regime transition
- Spikes → local instability

RSD localizes regime shifts **without forecasting**.

## Conclusion

RSD detects regime transitions by monitoring structural instability,
not by predicting future values.

This makes it suitable for:
- early diagnostics
- monitoring non-stationary systems
- validating regime assumptions
